In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

# Importing necessary models for implementation of ANN
from keras.models import Sequential
#, Activation,Layer,Lambda
from keras.layers import Dense 
#a=100
from sklearn.model_selection import train_test_split

In [6]:
wbcd = pd.read_csv('pima-indians-diabetes.csv')
#wbcd.drop(["id"],axis=1,inplace=True) # Dropping the uncessary column 
wbcd.columns
wbcd.shape

(767, 9)

## Check for Null values

In [7]:
# No missing values 
wbcd.isnull().sum().sum()

0

In [8]:
#  Malignant as 0 and Beningn as 1
wbcd.loc[wbcd.Outcome=="B","Outcome"] = 1
wbcd.loc[wbcd.Outcome=="M","Outcome"] = 0
wbcd.Outcome.value_counts().plot(kind="bar")

AttributeError: 'DataFrame' object has no attribute 'Outcome'

In [ ]:
train,test = train_test_split(wbcd,test_size = 0.3,random_state=42)

trainX = train.drop(["Outcome"],axis=1)
trainY = train["Outcome"]
testX = test.drop(["Outcome"],axis=1)
testY = test["Outcome"]
test.Outcome.value_counts()

In [ ]:
def prep_model(hidden_dim):
    model = Sequential() # initialize 
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],activation="relu"))
        else:
            model.add(Dense(hidden_dim[i],activation="relu"))
    # To define the dimensions for the output layer
    # activation - sigmoid 
    model.add(Dense(hidden_dim[-1],kernel_initializer="normal",activation="sigmoid"))
    # loss function -> loss parameter
    # algorithm to update the weights - optimizer parameter
    # accuracy - metric to display for 1 epoch
    model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])
    return model  

In [ ]:
model = Sequential() # initialize 
#Input Layer
model.add(Dense(12,input_dim=30,activation="relu"))
model.add(Dense(12,activation="relu"))
model.add(Dense(12,activation="relu"))
# To define the dimensions for the output layer
# activation - sigmoid 
model.add(Dense(1,kernel_initializer="normal",activation="sigmoid"))
# loss function -> loss parameter
# algorithm to update the weights - optimizer parameter
# accuracy - metric to display for 1 epoch
model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])

### First Model
number of input features - 30

number of hidden layers - 3  layers

number of hidden neurons in each hidden layer - 12

Number of dimensions for output layer  - 1 


In [ ]:
first_model = Sequential() # initialize 
#Input Layer
first_model.add(Dense(12,input_dim=8,activation="relu"))
first_model.add(Dense(12,activation="relu"))
first_model.add(Dense(12,activation="relu"))
# To define the dimensions for the output layer
# activation - sigmoid 
first_model.add(Dense(1,kernel_initializer="normal",activation="sigmoid"))
# loss function -> loss parameter
# algorithm to update the weights - optimizer parameter
# accuracy - metric to display for 1 epoch
first_model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])

In [ ]:
# Fitting ANN model with epochs = 100 
first_model.fit(np.array(trainX),np.array(trainY),epochs=10)

### Predicting for Train Data

In [ ]:
# predicting the probability values for each record
pred_train = first_model.predict(np.array(trainX))

# pd.Series - > convert list format Pandas Series data structure
pred_train = pd.Series([i[0] for i in pred_train])

disease_class = ["B","M"]
# converting series because add them as columns into data frame
pred_train_class = pd.Series(["M"]*398)
pred_train_class[[i>0.5 for i in pred_train]] = "B"
#pred_train_class
train.columns

In [ ]:
from sklearn.metrics import confusion_matrix
train["original_class"] = "M"
train.loc[train.diagnosis==1,"original_class"] = "B"
train.original_class.value_counts()

# Two way table format 
confusion_matrix(pred_train_class,train.original_class)
#train.columns

In [ ]:
# Calculating the accuracy using mean function from numpy 
# we need to reset the index values of train data as the index values are random numbers
np.mean(pred_train_class==pd.Series(train.original_class).reset_index(drop=True))

# 2 way table 
pd.crosstab(pred_train_class,pd.Series(train.original_class).reset_index(drop=True))

### Predicting for Test Data

In [ ]:
# Predicting for test data 
pred_test = first_model.predict(np.array(testX))

pred_test = pd.Series([i[0] for i in pred_test])
pred_test_class = pd.Series(["M"]*171)

pred_test_class[[i>0.5 for i in pred_test]] = "B"
test["original_class"] = "M"
test.loc[test.diagnosis==1,"original_class"] = "B"
test.original_class.value_counts()
temp = pd.Series(test.original_class).reset_index(drop=True)
np.mean(pred_test_class==pd.Series(test.original_class).reset_index(drop=True)) # 97.66
len(pred_test_class==pd.Series(test.original_class).reset_index(drop=True))
confusion_matrix(pred_test_class,temp)
#pd.crosstab(pred_test_class,test.original_class)

# to show the count of each category with respect to other category 
pd.crosstab(test.original_class,pred_test_class).plot(kind="bar")

## Predicting Continuous Values

In [ ]:
# Reading data 
Concrete = pd.read_csv("concrete.csv")
Concrete.head()
Concrete.shape

In [ ]:
column_names = list(Concrete.columns)
predictors = column_names[0:8]
target = column_names[8]
predictors

In [ ]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],kernel_initializer="normal",activation="relu"))
        else:
            model.add(Dense(hidden_dim[i],activation="relu"))
    # for the output layer we are not adding any activation function as 
    # the target variable is continuous variable 
    model.add(Dense(hidden_dim[-1]))
    # loss ---> loss function is means squared error to compare the output and estimated output
    # optimizer ---> adam
    # metrics ----> mean squared error - error for each epoch on entire data set 
    model.compile(loss="mean_squared_error",optimizer="adam",metrics = ["mse"])
    return (model)

In [ ]:
first_model = prep_model([8,50,1])
first_model.fit(np.array(Concrete[predictors]),np.array(Concrete[target]),epochs=20)

In [ ]:
first_model = prep_model([8,50,1])

In [ ]:
pred_train = first_model.predict(np.array(train[predictors]))
pred_train = pd.Series([i[0] for i in pred_train])
rmse_value = np.sqrt(np.mean((pred_train-train[target])**2))
rmse_value